In [1]:
import tensorflow as tf
import pandas as pd
import requests
import json
import base64

In [7]:
df = pd.read_csv("data/mental_health.csv")
test_text = df["text"][1]

In [8]:
test_text

'nothing look forward lifei dont many reasons keep going feel like nothing keeps going next day makes want hang myself'

In [9]:
def prepare_json(text):
    feature_spec = {
        "text": tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(text, "utf-8")]))
    }
    
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
    
    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]
    
    return json.dumps({
        "signature_name": "serving_default",
        "instances": result
    })

In [10]:
def make_predictions(text):
    json_data = prepare_json(text)
    
    endpoint = "http://localhost:8080/v1/models/mentalhealth-detection-model:predict"
    response = requests.post(endpoint, data=json_data)

    prediction = response.json()["predictions"][0][0]
    return "Komentar beracun terhadap mental health" if prediction > 0.5 else "Bukan Komentar beracun terhadap mental health"

In [11]:
make_predictions(test_text)

'Komentar beracun terhadap mental health'